In [ ]:
# default_exp environment

In [1]:
#export
import torch
import numpy as np
from math import pi, cos, sin
from tqdm.auto import tqdm

# Helpers 

In [57]:
#export
from scipy.optimize import curve_fit
from scipy.special import factorial
from scipy.stats import poisson


def get_distance(x):
    M = np.reshape(np.repeat(x[ :, :], x.shape[0], axis = 0), (x.shape[0], x.shape[0], 2))
    Mtrans = M.transpose(1,0,2)
    distance = np.sqrt(np.square(M[:,:, 0]-Mtrans[:,:, 0])
                     + np.square(M[:,:, 1]-Mtrans[:,:, 1]))  
    return distance

def isBetween(a, b, c, epsilon = 0):
    crossproduct = (c[1] - a[1]) * (b[0] - a[0]) - (c[0] - a[0]) * (b[1] - a[1])

    # compare versus epsilon for floating point values, or != 0 if using integers
    if abs(crossproduct) > epsilon:
        return False

    dotproduct = (c[0] - a[0]) * (b[0] - a[0]) + (c[1] - a[1])*(b[1] - a[1])
    if dotproduct < 0:
        return False

    squaredlengthba = (b[0] - a[0])*(b[0] - a[0]) + (b[1] - a[1])*(b[1] - a[1])
    if dotproduct > squaredlengthba:
        return False

    return True

def isBetween_ab_Vec(a, b, c, epsilon = 0):
    mask = np.array([True]*a.shape[0])
    
    crossproduct = (c[1] - a[:,1]) * (b[:,0] - a[:,0]) - (c[0] - a[:, 0]) * (b[:, 1] - a[:, 1])
    mask[np.argwhere(abs(crossproduct) > epsilon)] = False    

    dotproduct = (c[0] - a[:, 0]) * (b[:, 0] - a[:, 0]) + (c[1] - a[:, 1])*(b[:, 1] - a[:, 1])
    mask[np.argwhere(dotproduct < 0)] = False

    squaredlengthba = (b[:, 0] - a[:, 0])*(b[:, 0] - a[:, 0]) + (b[:, 1] - a[:, 1])*(b[:, 1] - a[:, 1])
    mask[np.argwhere(dotproduct > squaredlengthba)] = False
    
    return mask

def isBetween_c_Vec(a, b, c, epsilon = 0):
    mask = np.array([True]*c.shape[0])
    
    crossproduct = (c[:,1] - a[1]) * (b[0] - a[0]) - (c[:,0] - a[0]) * (b[1] - a[1])
    mask[np.argwhere(abs(crossproduct) > epsilon)] = False    

    dotproduct = (c[:, 0] - a[0]) * (b[0] - a[0]) + (c[:, 1] - a[1])*(b[1] - a[1])
    mask[np.argwhere(dotproduct < 0)] = False

    squaredlengthba = (b[0] - a[0])*(b[0] - a[0]) + (b[1] - a[1])*(b[1] - a[1])
    mask[np.argwhere(dotproduct > squaredlengthba)] = False
    
    return mask

def is_c_between_ab(a, b, c, radius = 1):    
    distances = np.abs(np.cross(b-a, a-c))/np.linalg.norm(b-a,axis=1)
    return distances < radius 
    

# Environment 

In [54]:
#export
class target_env():
    def __init__(self,
                 Nt = 1000,
                 L = 200,
                 at = 1,
                 ls = 2,
                 agent_step = 1,
                 boundary_condition = 'periodic'):
        
        self.Nt = Nt
        self.L = L
        self.at = at
        self.ls = ls
        self.boundary_condition = boundary_condition
        
        self.init_env()
        
        self.agent_step = agent_step 
                
    def init_env(self):
        self.target_positions = torch.rand(self.Nt, 2)*self.L
        self.agent_pos = torch.rand(2)*self.L
        self.previous_pos = self.agent_pos.clone()
        
          
    def step(self, direction):        
            
        # Save previous position to check if crossing happened
        self.previous_pos = self.agent_pos.clone()
            
        self.agent_pos[0] = self.agent_pos[0] + self.agent_step*cos(direction)
        self.agent_pos[1] = self.agent_pos[1] + self.agent_step*sin(direction)
        
        self.check_bc()
                
    def check_encounter(self):
        encounters = isBetween_c_Vec(self.agent_pos, self.previous_pos, self.target_positions, epsilon = self.at)
        
        if sum(encounters) > 0:   
            distance_previous_pos = torch.sqrt((self.agent_pos[0]-self.target_positions[:, 0])**2 + (self.agent_pos[1]-self.target_positions[:, 1])**2)            
            
            # checking which encountered point is closer to previous position
            min_distance_masked = torch.argmin(distance_previous_pos[encounters])
            first_encounter = torch.arange(self.Nt)[encounters][min_distance_masked]
            
            # If there was encounter, update the position to (pos target + ls)
            self.agent_pos[0] = self.target_positions[first_encounter, 0] + self.ls*np.cos(torch.rand(1)*2*pi)
            self.agent_pos[1] = self.target_positions[first_encounter, 1] + self.ls*np.sin(torch.rand(1)*2*pi)
                        
            self.check_bc()
            
            return 1
        
        else: return 0
        
    def check_bc(self): 
        if self.boundary_condition == 'reflectant':
            while torch.max(self.agent_pos) > self.L or torch.min(self.agent_pos)< 0: 
                out_high = self.agent_pos > self.L
                out_low = self.agent_pos < 0
                self.agent_pos[out_high] = self.agent_pos[out_high] - 2*(self.agent_pos[out_high] - self.L)
                self.agent_pos[out_low] = - self.agent_pos[out_low]


        elif self.boundary_condition == 'periodic':
            while torch.max(self.agent_pos) > self.L or torch.min(self.agent_pos)< 0: 
                out_high = self.agent_pos > self.L
                out_low = self.agent_pos < 0 
                self.agent_pos[out_high] = self.agent_pos[out_high] - self.L
                self.agent_pos[out_low] = self.L + self.agent_pos[out_low]  


In [60]:
#export
def target_search(Nt = 100, at = 1, ls = 1,
                  rho = 1e-3,
                  L= 1000,   
                  N = 1000, T = 1000,
                  sample_step = None,
                  **args_step):

    diag = np.sqrt(2*L**2)

    target_positions = np.random.rand(Nt, 2)*L

    pos = np.zeros((N, T, 2))
    pos[:, 0, :] = np.random.rand(N, 2)*L

    encounter = np.zeros((N, T)).astype(bool)
    
    real_steps = []
    sampled_steps = np.zeros((N,T))

    
    for t in tqdm(range(1,T)):
        
        angles = np.random.uniform(low = 0, high = 2*np.pi, size = N)
        
        if sample_step is None:
            steps = np.random.randn(N)
        else:
            steps = sample_step(**args_step)
            
        sampled_steps[:, t] = steps        
        
        pos[:, t, 0] = (pos[:, t-1, 0] + steps*np.cos(angles))%L
        pos[:, t, 1] = (pos[:, t-1, 1] + steps*np.sin(angles))%L            
        

        masked = np.array([False]*N)
        for idxT, pT in enumerate(target_positions):
            
            mask = is_c_between_ab(pos[:, t, :], pos[:, t-1, :], pT, radius = at)

            masked += np.array(mask)
            
            # Check real distance travelled
            if np.sum(mask) > 0:
                real_steps += np.abs(pos[mask == True, t, :].copy() - pT).tolist()[0]
            
        
            pos[mask == True, t, :] = pT
        
        encounter[:, t] = masked
        sum_m = np.sum(masked)

        '''
        See that we want to walk, not to fly, because it's closer to what RL will do
        '''
        if sum_m > 0:
            angle = np.random.uniform(low = 0, high = 2*np.pi, size = sum_m)
            pos[masked == True, t, 0] += ls*np.cos(angle)
            pos[masked == True, t, 1] += ls*np.sin(angle)

        
                
    return encounter, real_steps, sampled_steps
                

In [63]:
from nbdev.export import notebook2script
notebook2script()

Converted cluster_launch.ipynb.
Converted diffusion_walks.ipynb.
Converted environment.ipynb.
Converted theoretical_policy.ipynb.
